In [33]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
# Define the model class
class LinearRegressor(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegressor, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        return self.linear(x)

class LogisticRegression(torch.nn.Module):    
    # build the constructor
    def __init__(self, input_size, output_size):
        super().__init__()
        self.net = nn.Sequential( 
#           nn.Linear(input_size, input_size),
#           nn.ReLU(),
#           nn.Linear(input_size, input_size),
#           nn.ReLU(),
          nn.Linear(input_size, output_size),
          nn.ReLU()
        )
    # make predictions
    def forward(self, x):
        y_pred = torch.sigmoid(self.net(x))
        
        return y_pred

In [54]:
# Example Data (Replace with your actual data)
# X_train: tensor of shape [n_samples, n_features]
# y_train: tensor of shape [n_samples, 1]
#data generated from below
#https://drive.google.com/drive/folders/1z726l_xZuSZAAPpUSljA5YM-NlcDSR8S
#https://drive.google.com/drive/folders/1mC8UQJmgOXAD0eMCV8H0ZJBmc57KhYIp

directory = '/shared/share_mala/yuanzhe/adaptive_sampling/eicu/eicu_simple_bias/important_features/'
#below is biased training data
csv_file =  'eicu_random_prop_score_selected2_16.0_only_important_feature_simple.csv'
df = pd.read_csv(directory + csv_file)
X_col = list(df.columns)
X_col.remove('EVENT_LABEL')

X_train = np.array(df[X_col])
y_train = np.array(df[['EVENT_LABEL']])  

# Convert data to PyTorch tensors if they aren't already
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

# Hyperparameters
epochs = 100  # Number of training iterations
learning_rate = 0.01

# Model, Loss and Optimizer
model = LogisticRegression(input_size=X_train.shape[1], output_size=1)
criterion = nn.MSELoss()
criterion = torch.nn.BCELoss()
#optimizer = optim.SGD(model.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate, betas=(0.9, 0.999))

# Training loop
for epoch in range(epochs):
    # Forward pass: Compute predicted y by passing X to the model
    y_pred = model(X_train)

    # Compute and print loss
    loss = criterion(y_pred, y_train)
    if epoch % 10 == 0:  # Print every 10th epoch
        print(f'Epoch {epoch}, Loss: {loss.item()}')

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

#torch.save(model, url + 'predictor.pkl')

model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save(directory + 'predictor_eicu_0206.pt') # Save

#https://stackoverflow.com/questions/55488795/unpickling-saved-pytorch-model-throws-attributeerror-cant-get-attribute-net

Epoch 0, Loss: 61.822731018066406
Epoch 10, Loss: 0.6931473612785339
Epoch 20, Loss: 0.6931473612785339
Epoch 30, Loss: 0.6931473612785339
Epoch 40, Loss: 0.6931473612785339
Epoch 50, Loss: 0.6931473612785339
Epoch 60, Loss: 0.6931473612785339
Epoch 70, Loss: 0.6931473612785339
Epoch 80, Loss: 0.6931473612785339
Epoch 90, Loss: 0.6931473612785339


In [55]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def Recall_True(x_test, y_test, model, device): #input is dataloader_test and classifier/ model c, output is true recall given labels
 
    prediction_list = model(x_test)
    predicted_class = torch.argmax(prediction_list)
    predicted_class = prediction_list >= 0.5 #may need to use the previous code if model predicts probs of two classes

    x = torch.sum(torch.mul(y_test, predicted_class))
    y = torch.sum(y_test)
    return x/y

test_csv_name = '/shared/share_mala/yuanzhe/adaptive_sampling/eicu/EICU_train_test/eicu_test_final.csv'
df_test = pd.read_csv(test_csv_name)
X_test = np.array(df_test[X_col])
y_test = np.array(df_test[['EVENT_LABEL']]) 
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

print(model(X_test))
print('Recall on training data', Recall_True(X_train,y_train,model,device))
print('Recall on test data', Recall_True(X_test,y_test,model,device))

tensor([[0.5000],
        [0.5000],
        [0.5000],
        ...,
        [0.5000],
        [0.5000],
        [0.5000]], grad_fn=<SigmoidBackward0>)
Recall on training data tensor(1.)
Recall on test data tensor(1.)


In [18]:
#convert reg to classification, no need to run again
# url = '/shared/share_mala/yuanzhe/adaptive_sampling/pipeline_datasets/'
# train_csv_name = 'input_dim_1_train_init_data_mean_0.0ln_1.0sig_0.1no.2000.csv'
# test_csv_name = 'input_dim_1_test_final_data_mean_0.0ln_1.0sig_0.1no.2000.csv'
# pool_csv_name = 'input_dim_1_pool_data_mean_0.0ln_1.0sig_0.1no.2000.csv'
# file_list  = [train_csv_name, test_csv_name, pool_csv_name]
# #convert y into 0/1

# for f in file_list:
#     df = pd.read_csv(url + f)
#     df['EVENT_LABEL'] = df['EVENT_LABEL'] > 0
#     df.to_csv(url+'classifier_'+ f, index = False)
# #/user/ym2865/Adaptive Sampling/src